In [2]:
from census import Census
import os 

# Get Api key from user env
c_apikey = os.environ.get("CENSUS_APIKEY")

if c_apikey:
    c = Census(c_apikey)
else:
    print("User did not set CENSUS_APIKEY in their environment")

User did not set CENSUS_APIKEY in their environment
